<a href="https://colab.research.google.com/github/MatildaBeinat/KCL-Thesis-data-extraction/blob/main/Dimensions_Data_extraction_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install dimcli tqdm plotly -U --quiet

import dimcli
from dimcli.utils import *

import sys, time, json
import pandas as pd
from tqdm.notebook import tqdm as progressbar
import numpy as np
import ast

import plotly.express as px
if not 'google.colab' in sys.modules:
  # make js dependecies local / needed by html exports
  from plotly.offline import init_notebook_mode
  init_notebook_mode(connected=True)

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
  import getpass
  KEY = getpass.getpass(prompt='API Key: ')
  dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
  KEY = ""
  dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Feature selection

## Original dataset: feature selection

### Full
Chunk it due to size of records.


In [ ]:
df1 = dsl.query_iterative(f"""
                search publications
                in title_abstract_only for "dementia OR alzheimer*"
                where (research_org_countries = "GB"
                and year in [2000:2023])
                return publications[
                id+
                mesh_terms+
                category_hra+
                category_hrcs_rac+
                category_rcdc
                ]""").as_dataframe()

In [ ]:
df2 = dsl.query_iterative(f"""
                search publications
                in title_abstract_only for "dementia OR alzheimer*"
                where (research_org_countries = "GB"
                and year in [2000:2023])
                return publications[
                id+
                reference_ids+
                recent_citations+
                altmetric+
                relative_citation_ratio+
                times_cited+
                authors+
                authors_count+
                funder_countries+
                funders+
                journal+
                open_access+
                research_org_names+
                research_org_country_names+
                supporting_grant_ids
                ]""").as_dataframe()

In [ ]:
UK_dementia_publications = df1.merge(df2, on="id")
UK_dementia_publications.to_csv("/content/drive/MyDrive/Matilda thesis/UK dementia publications.csv")

In [ ]:
UK_dementia_publications = pd.read_csv("/content/drive/MyDrive/Matilda thesis/UK dementia publications.csv")
UK_dementia_publications.drop('Unnamed: 0', axis=1, inplace=True)

### Data wrangling

extract information about the author and research organisation

In [ ]:
# Count the number of IDs in the reference_ids column
UK_dementia_publications['reference_ids_count'] = UK_dementia_publications['reference_ids'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Drop the reference_ids column
UK_dementia_publications.drop('reference_ids', axis=1, inplace=True)


# Define a function to extract information about the first author
def extract_first_author_info(authors_info):
    if authors_info:
        first_author = authors_info[0]
        if first_author['affiliations']:
            first_affiliation = first_author['affiliations'][0]
            author_name = f"{first_author.get('first_name', '')} {first_author.get('last_name', '')}"
            return (first_author.get('researcher_id', None), author_name,
                    first_affiliation.get('id', None),
                    first_affiliation.get('name', None), first_affiliation.get('country', None))
        else:
            author_name = f"{first_author.get('first_name', '')} {first_author.get('last_name', '')}"
            return (first_author.get('researcher_id', None), author_name, None, None, None)
    else:
        return (None, None, None, None, None)

# Apply the function to the authors column to create new columns with first author information
(first_author_id, author_name,
 first_author_affiliation_id, first_author_affiliation_name, first_author_affiliation_country) = \
    zip(*UK_dementia_publications['authors'].apply(extract_first_author_info))

UK_dementia_publications['first_author_id'] = first_author_id
UK_dementia_publications['Author_name'] = author_name
UK_dementia_publications['first_author_affiliation_id'] = first_author_affiliation_id
UK_dementia_publications['first_author_affiliation_name'] = first_author_affiliation_name
UK_dementia_publications['first_author_affiliation_country'] = first_author_affiliation_country

# Drop the authors column
UK_dementia_publications.drop('authors', axis=1, inplace=True)

#Count number of countries
# Check if 'research_org_country_names' is a list
if isinstance(UK_dementia_publications['research_org_country_names'].iloc[0], list):
    # If it's a list, count the number of elements in the list
    UK_dementia_publications['count_research_org_country_names'] = UK_dementia_publications['research_org_country_names'].apply(len)
else:
    # If it's a string (country names are separated by a delimiter such as comma), split the string by the delimiter and count the number of elements
    UK_dementia_publications['count_research_org_country_names'] = UK_dementia_publications['research_org_country_names'].apply(lambda x: len(x.split(',')))


#Count number of research_orgs
# Check if 'research_org_names' is a list
if isinstance(UK_dementia_publications['research_org_names'].iloc[0], list):
    # If it's a list, count the number of elements in the list
    UK_dementia_publications['count_research_org_names'] = UK_dementia_publications['research_org_names'].apply(len)
else:
    # If it's a string (country names are separated by a delimiter such as comma), split the string by the delimiter and count the number of elements
    UK_dementia_publications['count_research_org_names'] = UK_dementia_publications['research_org_names'].apply(lambda x: len(x.split(',')))

## PUBLICATIONS cited by patents

### Adding number of patent citations to UK_dementia_publications

In [ ]:
# Create a list of your publications
pubsids = list(UK_dementia_publications["id"])

# Create a query and run through publications in chunks, there are too many results to process at once otherwise
pubs_chunks = dsl.query_iterative(f"""
search publications
in title_abstract_only for "dementia OR alzheimer*"
where (research_org_countries = "GB"
and year in [2000:2023])
return publications[id]""").chunks(500)

# Search for patents for the publications in the list created
query_results = []

for c in pubs_chunks:

      pubslist = json.dumps(list(pd.DataFrame(c).id))

      query_results.append(

                  dsl.query_iterative(f"""
                        search patents
                            where publication_ids in {pubslist}
                            return patents[basics+publication_ids]
                        """).as_dataframe()
      )

# Concatenate the patents to the publication list based on the id
Patents_citing_UK_pubs = pd.concat(query_results).\
   drop_duplicates(subset='id')

if 'publication_ids' in Patents_citing_UK_pubs:
    # turning lists into strings to ensure compatibility with CSV loaded data
    # see also: https://stackoverflow.com/questions/23111990/pandas-dataframe-stored-list-as-string-how-to-convert-back-to-list
    Patents_citing_UK_pubs['publication_ids'] = Patents_citing_UK_pubs['publication_ids'].apply(lambda x: ','.join(map(str, x)))
else:
    Patents_citing_UK_pubs['publication_ids'] = ""

# Count patents per grant and enrich the original dataset
def patents_citing_pubsids(pubsids):
  global Patents_citing_UK_pubs
  return Patents_citing_UK_pubs[Patents_citing_UK_pubs['publication_ids'].str.contains(pubsids)]

print("===\nCounting patents citations per publication...")

l = []
for x in progressbar(pubsids):
  l.append(len(patents_citing_pubsids(x)))

UK_dementia_publications['Citing patents'] = l

print("===\nDone")

In [ ]:
UK_dementia_publications.to_csv("/content/drive/MyDrive/Matilda thesis/UK dementia publications.csv")
UK_dementia_publications.columns

In [ ]:
UK_dementia_publications= pd.read_csv("/content/drive/MyDrive/Matilda thesis/UK dementia publications.csv")
UK_dementia_publications.drop('Unnamed: 0', axis=1, inplace=True)

To check which publications are cited by patents and those not cited by patents use code below, this can also be checked in excel using the filter option

In [ ]:
UK_publications_cited_by_patents = UK_dementia_publications[UK_dementia_publications['Citing patents'] > 0] # Dataframe with "Citing Patents" > 0
UK_publications_NOT_cited_by_patents= UK_dementia_publications[UK_dementia_publications['Citing patents'] == 0] # Dataframe with "Citing Patents" = 0

In [ ]:
print(len(UK_publications_cited_by_patents))
print(len(UK_publications_NOT_cited_by_patents))

##PUBLICATIONS cited by clinical trials

###Add associated trials to each publication in the UK

In [ ]:
# pull out the grant IDs as a list
pubsids = list(UK_dementia_publications["id"])

# Create a query and run through publications in chunks, there are too many results to process at once otherwise
pubs_chunks_CT = dsl.query_iterative(f"""
search publications
in title_abstract_only for "dementia OR alzheimer*"
where (research_org_countries = "GB"
and year in [2000:2023])
return publications[id]""").chunks(500)

# Search for patents for the publications in the list created
query_results_UK_with_CT_pubs = []

for c in pubs_chunks_CT:

      UK_pubsids_with_CT = json.dumps(list(pd.DataFrame(c).id))

      query_results_UK_with_CT_pubs.append(

                  dsl.query_iterative(f"""
                        search clinical_trials
                            where publication_ids in {UK_pubsids_with_CT}
                            return clinical_trials[basics+publication_ids]
                        """).as_dataframe()
      )

# Concatenate the patents to the publication list based on the id
CT_citing_UK_pubs = pd.concat(query_results_UK_with_CT_pubs).\
   drop_duplicates(subset='id')

if 'publication_ids' in CT_citing_UK_pubs:
    # turning lists into strings to ensure compatibility with CSV loaded data
    # see also: https://stackoverflow.com/questions/23111990/pandas-dataframe-stored-list-as-string-how-to-convert-back-to-list
    CT_citing_UK_pubs['publication_ids'] = CT_citing_UK_pubs['publication_ids'].apply(lambda x: ','.join(map(str, x)))
else:
    CT_citing_UK_pubs['publication_ids'] = ""

# count patents per grant and enrich the original dataset
def CT_citing_pubsids(pubsids):
  global CT_citing_UK_pubs
  return CT_citing_UK_pubs[CT_citing_UK_pubs['publication_ids'].str.contains(pubsids)]

print("===\nCounting CT citations per publications...")

l = []
for x in progressbar(pubsids):
  l.append(len(CT_citing_pubsids(x)))

UK_dementia_publications['Associated trials'] = l

print("===\nDone")

To check which publications are cited by clinical trials and those not cited by patents use code below, this can also be checked in excel using the filter option

In [ ]:
UK_dementia_pubs_with_CT = UK_dementia_publications[UK_dementia_publications['Associated trials'] > 0] # Dataframe with "Citing Patents" > 0
UK_dementia_pubs_NO_CT= UK_dementia_publications[UK_dementia_publications['Associated trials'] == 0] # Dataframe with "Citing Patents" = 0

In [ ]:
print(UK_dementia_pubs_with_CT.shape)
print(UK_dementia_pubs_NO_CT.shape)

## add labels

In [ ]:
#To add columns for your target label, df is your features dataframe WITH citing patents and Associated trials.
# Assuming df is your DataFrame and 'Citing patents' and 'associated trials' are your columns
#df['label_patents'] = df['Citing patents'].apply(lambda x: 1 if x > 0 else 0)
#df['label_trials'] = df['Associated trials'].apply(lambda x: 1 if x > 0 else 0)

# Concepts

Create concepts dataframe with concepts scores

In [ ]:
# Search for concepts and their scores
Concepts_score = dsl.query_iterative(f"""
search publications
in title_abstract_only for "dementia OR alzheimer*"
where (research_org_countries = "GB"
and year in [2000:2023])
return publications[id+concepts_scores]""").as_dataframe()
Concepts_score = pd.DataFrame(Concepts_score)

In [ ]:
# work with a copy of the results so you keep the original
df1 = Concepts_score.copy()

# this checks each row in df1, and drops any row that is not in a list format, and it resets the index if it drops a row
i = 0
while i < 38641:
  if type(df1["concepts_scores"].iloc[i]) is not list:
    df1.drop([i], axis=0, inplace=True)
  i+=1

df1.reset_index(inplace=True)
df1.drop(['index'], axis=1, inplace=True)
df1


In [ ]:
# Function that generate a list of concepts where concept relevance is greater than 0.5
# Param index - Index is the index in the dataframe
def remove_below_50(index):
  final = []
  scores_lst = df1["concepts_scores"].iloc[index]
  if type(scores_lst) is not list:
    df1.at[index,"concepts_scores"] = final
    return final
  for i in scores_lst:
    if i['relevance'] >= 0.5:
      final.append(i['concept'])

  df1.at[index,"concepts_scores"] = final
  return final

for i in range(len(df1)):
  remove_below_50(i)

# finds lists that are length 0, removes them
lst = []
i = 0
while i < len(df1):
  if len(df1["concepts_scores"].iloc[i])==0:
    lst.append(i)
  i+=1

df1.drop(lst, axis=0, inplace=True)
df1.reset_index(inplace=True)
df1.drop(['index'], axis=1, inplace=True)
df1

#this rename the columns from concepts_scores to just concepts
df1.rename(columns = {'concepts_scores':'concepts'}, inplace = True)

In [ ]:
df1.to_csv("/content/drive/MyDrive/Matilda thesis/official/concepts list with relevance.csv")

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/concepts list with relevance.csv")
df1.drop(['Unnamed: 0'], axis=1, inplace=True)

#this removes all the unnecessary values in the dataframe
for i in range(len(df1)):
  lst = df1["concepts"].iloc[i].strip('][').replace("\'","").replace("\"","").split(', ')
  df1.at[i,"concepts"] = lst

df1

up to here we have a dataframe with concepts that have relevance above .5, no empty cells.

finding all the unique concepts in df1 - dataframe with removed concepts scores below .5

In [ ]:
data = df1.copy()
# search for all the unique concepts in the df1 dataframe
unique_concepts = set()
for concepts in data['concepts']:
  for i in concepts:
    unique_concepts.add(i)

unique_concepts = list(unique_concepts)
print(len(unique_concepts))

2003


go through unique concepts, check how often they appear

In [ ]:
count_dict = {}

for i in unique_concepts:
  count_dict[i] = 0

print(len(count_dict))

for i in range(len(df1)):
  for j in df1["concepts"].iloc[i]:
    count_dict[j] +=1

2003


In [ ]:
concepts_count_prevalence = pd.DataFrame.from_dict(count_dict, orient='index')
concepts_count_prevalence.to_csv("/content/drive/MyDrive/Matilda thesis/official/concepts count prevalence.csv")

In [ ]:
concepts_count_prevalence = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/concepts count prevalence.csv")

remove all concepts that appear less than 20 times

In [ ]:
higher_prevalence = concepts_count_prevalence.copy()
higher_prevalence.rename(columns = {'Unnamed: 0':'Concept'}, inplace = True)
higher_prevalence.rename(columns = {'0':'Count'}, inplace = True)

higher_prevalence

In [ ]:
# remove all concepts that appear less than 20 times
lst = []
for i in range(len(higher_prevalence)):
  if higher_prevalence["Count"].iloc[i] < 20:
    lst.append(i)

higher_prevalence.drop(lst, axis=0, inplace=True)
higher_prevalence.reset_index(inplace=True)
higher_prevalence.drop(['index'], axis=1, inplace=True)

print(len(higher_prevalence))
higher_prevalence

In [ ]:
lst_concepts = higher_prevalence["Concept"].values.tolist() #Turn Concept column in dataframe to a list

for i in range(len(lst_concepts)):
  better = lst_concepts[i].replace("\'","")
  lst_concepts[i] = better

In [ ]:
# replace the original dataframe with the new dataframe of only concepts that appear more than 20 (or 5) times and have relevance of more than 0.5
for i in range(len(df1)):
  lst = df1["concepts"].iloc[i]
  lst2 = []
  for j in range(len(lst)):
    if lst[j] in lst_concepts:
      lst2.append(lst[j])
  df1.at[i,"concepts"] = lst2

In [ ]:
df1.to_csv("/content/drive/MyDrive/Matilda thesis/official/concepts appearance minimum 20.csv")

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/concepts appearance minimum 20.csv")
df1.drop(['Unnamed: 0'], axis=1, inplace=True)

for i in range(len(df1)):
  lst = df1["concepts"].iloc[i].strip('][').replace("\'","").replace("\"","").split(', ')
  df1.at[i,"concepts"] = lst


one hot encode df1 - for concepts prevalence 20 and relevance 0.5

In [ ]:
# Assuming df is your dataframe
df = df1.copy()

# Using the explode method, create a new dataframe where each concept is a separate row
df_exploded = df.explode('concepts')

# Strip any leading/trailing white space from the concepts
df_exploded['concepts'] = df_exploded['concepts'].str.strip()

# One-hot encode the exploded dataframe and group by id, using max as the aggregation function to avoid duplicate rows
one_hot = pd.get_dummies(df_exploded, columns=['concepts']).groupby('id', as_index=False).max()

# Display the resulting dataframe
print(one_hot)


In [ ]:
one_hot.to_csv("/content/drive/MyDrive/Matilda thesis/official/concepts one hot.csv")

In [ ]:
one_hot = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/concepts one hot.csv")

In [ ]:
one_hot1 = one_hot.copy()
one_hot1.rename(columns=lambda x: x[9:], inplace=True)

#ML data preparation

## add the year to the dataframe

In [ ]:
years = dsl.query_iterative(f"""
search publications
in title_abstract_only for "dementia OR alzheimer*"
where (research_org_countries = "GB"
and year in [2000:2023])
return publications[id+year]
""").as_dataframe()

In [ ]:
years = pd.DataFrame(years)

In [ ]:
years.to_csv("/content/drive/MyDrive/Matilda thesis/official/id + years.csv")

In [ ]:
years = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/id + years.csv")
years['id'] = years['id'].apply(lambda x: x[4:])

merge years with entire feature table

In [ ]:
Features_total = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/Features 2000-2023 clean for ML with labels.csv")


In [ ]:
Features_years_total = Features_total.merge(years, on='id')
Features_years_total.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=True)

In [ ]:
Features_years_total.to_csv("/content/drive/MyDrive/Matilda thesis/official/Features for ML with years.csv")

## Category_hrcs_rac cleaning

In [ ]:
Features_2000_2023 = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/UK dementia publications (5 - edited).csv")

In [ ]:
category_hrcs_rac = Features_2000_2023[["id", "category_hrcs_rac"]]
category_hrcs_rac.columns

In [ ]:
df = category_hrcs_rac

# Replace NaN values with '[]'
df.loc[:, 'category_hrcs_rac'] = df['category_hrcs_rac'].fillna('[]')

# Convert string representation of list of dicts to actual list of dicts
df.loc[:, 'category_hrcs_rac'] = df['category_hrcs_rac'].apply(ast.literal_eval)

# Extract the 'name' from each dictionary, remove the number prefix and join them with a comma
df.loc[:, 'category_hrcs_rac'] = df['category_hrcs_rac'].apply(
    lambda x: ', '.join([i['name'].split(' ', 1)[-1] for i in x])
)

print(df)


In [ ]:
df = category_hrcs_rac

In [ ]:
category_hrcs_rac.to_csv("/content/drive/MyDrive/Matilda thesis/category hrcs rac + ID.csv")

merge with feature list

In [ ]:
Features_2000_2023_racclean = Features_2000_2023.merge(category_hrcs_rac, on="id")

In [ ]:
Features_2000_2023_racclean.to_csv("/content/drive/MyDrive/Matilda thesis/Features 2000-2023 with hrcs_rac cleaned.csv")

In [ ]:
Features_2000_2023_racclean = pd.read_csv("/content/drive/MyDrive/Matilda thesis/Features 2000-2023 with hrcs_rac cleaned.csv")

### one hot encode rac

In [ ]:
id_rac = Features_total[['id', 'category_hrcs_rac_clean']]

In [ ]:
id_rac.head()

In [ ]:
id_rac.to_csv("/content/drive/MyDrive/Matilda thesis/official/id and rac.csv")

In [ ]:
id_rac = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/id and rac.csv")
id_rac.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)
id_rac

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your dataframe
df = id_rac.copy()

# Fill NaN values with an empty string
df['category_hrcs_rac_clean'] = df['category_hrcs_rac_clean'].fillna('')

# Split the concepts on commas and create a list of concepts
df['category_hrcs_rac_clean'] = df['category_hrcs_rac_clean'].apply(lambda x: x.split(','))

# Using the explode method, create a new dataframe where each concept is a separate row
df_exploded = df.explode('category_hrcs_rac_clean')

# Strip any leading/trailing white space from the concepts
df_exploded['category_hrcs_rac_clean'] = df_exploded['category_hrcs_rac_clean'].str.strip()

# One-hot encode the exploded dataframe and group by id, using max as the aggregation function to avoid duplicate rows
one_hot_rac = pd.get_dummies(df_exploded, columns=['category_hrcs_rac_clean']).groupby('id', as_index=False).max()

# Display the resulting dataframe
print(one_hot_rac)

In [ ]:
one_hot_rac.to_csv("/content/drive/MyDrive/Matilda thesis/official/one hot encoding rac.csv")

In [ ]:
one_hot_rac = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/one hot encoding rac.csv")

## Category_hra cleaning

In [ ]:
category_hra= Features_2000_2023_racclean[["id", "category_hra"]]
category_hra.columns

Index(['id', 'category_hra'], dtype='object')

In [ ]:
import pandas as pd
import numpy as np
import ast

# First, we'll convert the string representations of lists/dicts into actual lists/dicts
category_hra['category_hra'] = category_hra['category_hra'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)

# Now, let's extract the 'name' key from each dictionary in the 'category_hra' column
category_hra['category_hra'] = category_hra['category_hra'].apply(lambda x: x[0]['name'] if pd.notnull(x) else np.nan)

# print the DataFrame
print(category_hra)


In [ ]:
category_hra.to_csv("/content/drive/MyDrive/Matilda thesis/category hra + ID.csv")

merge with feature list

In [ ]:
Features_2000_2023_rac_hra_clean = Features_2000_2023_racclean.merge(category_hra, on="id")

In [ ]:
Features_2000_2023_rac_hra_clean.to_csv("/content/drive/MyDrive/Matilda thesis/Features 2000-2023 with hrcs_rac and hra cleaned.csv")

## Category_rcdc cleaning

In [ ]:
category_rcdc= Features_2000_2023_rac_hra_clean[["id", "category_rcdc"]]
category_rcdc.columns
len(category_rcdc)

Index(['id', 'category_rcdc'], dtype='object')

In [ ]:
import pandas as pd
import ast
import numpy as np

# Fill NaN values with '[]'
category_rcdc['category_rcdc'] = category_rcdc['category_rcdc'].fillna('[]')

# Convert string representation of list of dicts to actual list of dicts
category_rcdc['category_rcdc'] = category_rcdc['category_rcdc'].apply(ast.literal_eval)

# Extract the 'name' value from each dict and join with comma
category_rcdc['category_rcdc'] = category_rcdc['category_rcdc'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

print(category_rcdc)


In [ ]:
category_rcdc.to_csv("/content/drive/MyDrive/Matilda thesis/category rcdc + ID.csv")

merge with features list

In [ ]:
Features_2000_2023_rac_hra_clean = pd.read_csv("/content/drive/MyDrive/Matilda thesis/Features 2000-2023 with hrcs_rac and hra cleaned.csv")

In [ ]:
Features_2000_2023_rac_hra_rcdc_clean = Features_2000_2023_rac_hra_clean.merge(category_rcdc, on="id")

In [ ]:
Features_2000_2023_rac_hra_rcdc_clean.to_csv("/content/drive/MyDrive/Matilda thesis/Features 2000-2023 with hrcs_rac, hra and rcdc cleaned.csv")

## one hot encode rcdc

In [ ]:
id_rcdc = Features_total[['id', 'category_rcdc_clean']]

In [ ]:
id_rcdc.to_csv("/content/drive/MyDrive/Matilda thesis/official/id and rcdc.csv")

In [ ]:
id_rcdc = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/id and rcdc.csv")
id_rcdc.drop(['Unnamed: 0'], axis=1, inplace=True)
id_rcdc

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your dataframe
df = id_rcdc.copy()

# Fill NaN values with an empty string
df['category_rcdc_clean'] = df['category_rcdc_clean'].fillna('')

# Split the concepts on commas and create a list of concepts
df['category_rcdc_clean'] = df['category_rcdc_clean'].apply(lambda x: x.split(','))

# Using the explode method, create a new dataframe where each concept is a separate row
df_exploded = df.explode('category_rcdc_clean')

# Strip any leading/trailing white space from the concepts
df_exploded['category_rcdc_clean'] = df_exploded['category_rcdc_clean'].str.strip()

# One-hot encode the exploded dataframe and group by id, using max as the aggregation function to avoid duplicate rows
one_hot_rcdc = pd.get_dummies(df_exploded, columns=['category_rcdc_clean']).groupby('id', as_index=False).max()

# Display the resulting dataframe
print(one_hot_rcdc)

In [ ]:
one_hot_rcdc.to_csv("/content/drive/MyDrive/Matilda thesis/official/one hot encoding rcdc.csv")

## adding year published to dataframe

In [ ]:
pubs_cited = dsl.query_iterative(f"""
search publications
in title_abstract_only
for "dementia OR alzheimer*"
where (research_org_countries = "GB"
and year in [2000:2023])
return publications[id+times_cited+year]""").as_dataframe()
pubs_cited = pd.DataFrame(pubs_cited)

In [ ]:
pubs_cited.to_csv("/content/drive/MyDrive/Matilda thesis/check pubs cited.csv")

In [ ]:
Features = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/Features 2000-2023 clean for ML with labels.csv")

In [ ]:
pubs_cited

In [ ]:
pubs_cited = pd.read_csv("/content/drive/MyDrive/Matilda thesis/check pubs cited.csv")
pubs_cited.drop('Unnamed: 0', axis=1, inplace=True)
pubs_cited['id'] = pubs_cited['id'].apply(lambda x: x[4:])

In [ ]:
Id_label = Features[['id', 'Citing patents']]
Id_label.head()

In [ ]:
pubs_cited['id'] = pubs_cited['id'].astype(str)
pubs_cited['times_cited'] = pubs_cited['times_cited'].astype(str)
pubs_cited['year'] = pubs_cited['year'].astype(str)
Id_label['id'] = Id_label['id'].astype(str)
Id_label['Citing patents'] = Id_label['Citing patents'].astype(str)

In [ ]:
label_timescited = pubs_cited.merge(Id_label, on='id')
label_timescited

In [ ]:
label_timescited.to_csv("/content/drive/MyDrive/Matilda thesis/official/check label and times cited.csv")

## TSVD concepts

In [ ]:
sparse_matrix = csr_matrix(one_hot1)
n_components = 20  # Choose thenumber of components based on your requirements
svd = TruncatedSVD(n_components=n_components)
embeddings = svd.fit_transform(sparse_matrix)
embeddings.shape

In [ ]:
embeddings = pd.DataFrame(embeddings)
embeddings['id'] = df1['id']
embeddings['id'] = embeddings['id'].apply(lambda x: x[4:])

In [ ]:
embeddings.to_csv("/content/drive/MyDrive/Matilda thesis/official/TSVD.csv")

## t-sne concepts

In [ ]:
# Apply t-SNE to the dataset
tsne = TSNE(n_components=3, init="random")
tsne_result = tsne.fit_transform(one_hot1)
print(tsne_result.shape)
print(tsne_result)
print(type(tsne_result.shape))

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/Features 2000-2023 clean for ML patents (3).csv")

In [ ]:
label_id = df2[['id', 'Label']]
label_id

label_id['id'] = label_id['id'].apply(lambda x: str(x))

In [ ]:
a,b,c = zip(*tsne_result)

data = df1.copy()

data.drop(['concepts'], axis=1,inplace=True)

a = list(a)
b = list(b)
c = list(c)

data['a'] =  a
data['b'] =  b
data['c'] =  c

data['id'] = data['id'].apply(lambda x: x[4:])

data = data.merge(label_id, on='id')
data.to_csv("/content/drive/MyDrive/Matilda thesis/official/data concepts tsne ML.csv")

In [ ]:
data1 = data.copy()
data1.drop(['id'],axis=1,inplace=True)
data1

## t-sne category rcdc

In [ ]:
one_hot_rcdc = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/one hot encoding rcdc.csv")
one_hot_rcdc.rename(columns=lambda x: x[20:], inplace=True)
one_hot_rcdc.drop(['','',''],axis=1,inplace=True)
one_hot_rcdc.columns

In [ ]:
# Apply t-SNE to the dataset
tsne_rcdc = TSNE(n_components=3, init="random")
tsne_result_rcdc = tsne.fit_transform(one_hot_rcdc)
tsne_result_rcdc.shape

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/Features 2000-2023 clean for ML patents (3).csv")

In [ ]:
label_id = df2[['id', 'Label']]
label_id

label_id['id'] = label_id['id'].apply(lambda x: str(x))

In [ ]:
rcdc_nested = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/id and rcdc.csv")

In [ ]:
l,m,n = zip(*tsne_result_rcdc)

data = rcdc_nested.copy()

data.drop(['category_rcdc_clean'], axis=1,inplace=True)

a = list(l)
b = list(m)
c = list(n)

data['l'] =  l
data['m'] =  m
data['n'] =  n


rcdc_nested['id'] = rcdc_nested['id'].astype(str)
rcdc_nested['category_rcdc_clean'] = rcdc_nested['category_rcdc_clean'].astype(str)
label_id['Label'] = label_id['Label'].astype(str)
label_id['id'] = label_id['id'].astype(str)

data = data.merge(label_id, on='id')
data.to_csv("/content/drive/MyDrive/Matilda thesis/official/category rcdc tsne ML.csv")

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/category rcdc tsne ML.csv")
data.drop(['Unnamed: 0', 'Label', 'Unnamed: 0.1'],axis=1,inplace=True)
data

In [ ]:
data1 = data.copy()
data1.drop(['id', 'Unnamed: 0'],axis=1,inplace=True)
data1

# Check of data after cleaning

## Features check

In [ ]:
PUB_ID= "pub.1131066542"

q = dsl.query(f"""
search publications
where id = "{PUB_ID}"
return publications[basics+id+category_hrcs_rac+category_hra+category_rcdc+mesh_terms+authors_count+open_access+recent_citations+times_cited+altmetric+supporting_grant_ids+relative_citation_ratio]""").as_dataframe()
q

## concepts check

In [ ]:
PUB_ID= "pub.1029063222"

q = dsl.query(f"""
search publications
where id = "{PUB_ID}"
return publications[id+title+abstract+concepts_scores]""").as_dataframe()
q

## hrcs_rac check

In [ ]:
PUB_ID= "pub.1090797659"

q = dsl.query(f"""
search publications
where id = "{PUB_ID}"
return publications[id+category_hrcs_rac]""").as_dataframe()
q

## hra check

In [ ]:
PUB_ID= "pub.1020401106"

q = dsl.query(f"""
search publications
where id = "{PUB_ID}"
return publications[id+category_hra]""").as_dataframe()
q

## rcdc check

In [ ]:
PUB_ID= "pub.1020401106"

q = dsl.query(f"""
search publications
where id = "{PUB_ID}"
return publications[id+category_rcdc]""").as_dataframe()
q

## concepts NESTED with label

With the cleaned features file, create a label column based on citing patents and associated trials: if there are 0 patents, the label is 0, if there are more than 0 patents, the label is 1. The same goes for clinical trials

In [ ]:
Features_label_patentsML = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/Features 2000-2023 clean for ML patents (3).csv")

In [ ]:
# select only the id and the label from the features list
ID_label = Features_label_patentsML[["id", "Label"]]
ID_label.columns

Index(['id', 'Label'], dtype='object')

In [ ]:
# convert the ID_label into a string
ID_label['id'] = ID_label['id'].astype(str)
len(ID_label)

In [ ]:
Concepts_nested = pd.read_csv("/content/drive/MyDrive/Matilda thesis/official/concepts nested 2000-2023 3.csv")

In [ ]:
Concepts_nested.columns

Index(['id', 'concepts'], dtype='object')

In [ ]:
Concepts_nested['id'] = Concepts_nested['id'].astype(str)

In [ ]:
Concepts_with_label = Concepts_nested.merge(ID_label, on="id")

In [ ]:
Concepts_with_label.to_csv("/content/drive/MyDrive/Matilda thesis/official/Concepts with label 2000-2023.csv")